# Classification and Calibration Doings

In [4]:
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from pyqtgraph import PlotWidget, plot
import pyqtgraph as pg
import sys  # We need sys so that we can pass argv to QApplication
from DAQ import DAQ
import nidaqmx
from nidaqmx.stream_readers import AnalogMultiChannelReader
from nidaqmx.constants import AcquisitionType
import numpy as np

import time


class calibration_mode(QWidget):
    """
    This is my widget.
    """
    
    def __init__(self):
        super().__init__()
        
        self.global_layout = QHBoxLayout(self)
        
        self.daq=DAQ()
        
        #addons container
        containeradd=QGroupBox("Extra Controls")
        self.layout1=QVBoxLayout(containeradd)
        self.global_layout.addWidget(containeradd)
        
        #graph container
        containergraph=QGroupBox("Data Recording")
        self.layout2=QVBoxLayout(containergraph)
        self.global_layout.addWidget(containergraph)
        
        # graph itself
        self.layout3=QVBoxLayout(self)
        
        self.graphWidget = pg.PlotWidget()
        
        self.layout3.addWidget(self.graphWidget)

        self.x,self.y,self.sum = np.array([]),np.array([]),np.array([])
        
        self.graphWidget.setBackground('w')
        
        self.graphWidget.showGrid(x = True, y = True, alpha = 1.0)
        
        #self.graphWidget.setXRange(0,1,update=True)
        #self.graphWidget.setYRange(0,1,update=True)

        pen = pg.mkPen(color=(255, 0, 0))
        pen1= pg.mkPen(color=(0, 255, 0))
        pen2= pg.mkPen(color=(0, 0, 255))
        self.data_line =  self.graphWidget.plot(self.x, pen=pen)
        self.data_line1 =  self.graphWidget.plot(self.y, pen=pen1)
        self.data_line2 =  self.graphWidget.plot(self.sum, pen=pen2)
        
        self.layout2.addLayout(self.layout3)
        
        #buttons
        self.layout_buttons=QHBoxLayout(self)
        
        self.acq_button=QToolButton(self)
        self.acq_button.setText('Start Tracking')
        
        self.reset_button=QToolButton(self)
        self.reset_button.setText('Reset')
        
        self.acq_button.clicked.connect(self.button_pressed)
        self.layout_buttons.addWidget(self.acq_button)
        self.reset_button.clicked.connect(self.reset_d)
        self.layout_buttons.addWidget(self.reset_button)
        
        
        
        self.layout2.addLayout(self.layout_buttons)
        
    def thread_func(self):
        self.thread=QThread()
        self.worker=Worker_cal(10,10,'trips')
        self.worker.moveToThread(self.thread)
        self.thread.started.connect(self.worker.start_tracking)
        self.worker.finished.connect(self.thread.quit)
        self.worker.finished.connect(self.worker.deleteLater)
        self.thread.finished.connect(self.thread.deleteLater)
        self.worker.data.connect(self.update_plot_data)
        self.thread.start()
    
    def update_plot_data(self,data_retrieved):
        self.x=data_retrieved[0]
        self.y=data_retrieved[1]
        self.sum=data_retrieved[2]
        #self.x=np.concatenate((self.x,data_retrieved[0]),axis=0)
        #self.y=np.concatenate((self.y,data_retrieved[1]),axis=0)
        #self.sum=np.concatenate((self.sum,data_retrieved[2]),axis=0)
        
        self.data_line.setData(self.x)
        self.data_line1.setData(self.y)
        self.data_line2.setData(self.sum)# Update the data
        
    def reset_d(self):
        self.x,self.y,self.sum = np.array([]),np.array([]),np.array([])
        self.data_line.setData(self.x)
        self.data_line1.setData(self.y)
        self.data_line2.setData(self.sum)
    
    def button_pressed(self):
        if self.acq_button.text()=='Stop Tracking':
            self.acq_button.setText('Start Tracking')
            self.daq.task.close()
            self.worker.stop()
            self.thread.exit()
        else:
            self.daq=DAQ()
            self.thread_func()
            self.acq_button.setText('Stop Tracking')
            print(self.acq_button.text())
        
    
    def closeEvent(self, event):
        print(len(self.x))
        self.worker.stop()
        self.thread.exit()
        event.accept()
        
class Worker_cal(QThread):
    finished=pyqtSignal()
    data=pyqtSignal(np.ndarray)
    
    def __init__(self,acq,sr,file):
        super(Worker_cal,self).__init__()
        self.daq=DAQ()
        self.acq=acq
        self.sr=sr*1000
        self.file=file
    
    def callback(self,task_handle, every_n_samples_event_type,number_of_samples, callback_data):
        #print(every_n_samples_event_type)
        data_out = np.empty(shape=(3, number_of_samples))
        self.reader.read_many_sample(data_out,number_of_samples)
        self.data.emit(data_out)
        return 0
            #self.file_save(np.array(data_out),self.file)
    
    def start_tracking(self):
        #self.daq.task.start()
        n_points=self.sr*self.acq
        self.daq.task.timing.cfg_samp_clk_timing(rate=self.sr,
                                                 sample_mode=AcquisitionType.CONTINUOUS,
                                                samps_per_chan=n_points)
        
        #self.daq.task.in_stream.input_buf_size = 10
        self.reader = AnalogMultiChannelReader(self.daq.task.in_stream)
        self.daq.task.register_every_n_samples_acquired_into_buffer_event(800,self.callback)
        self.daq.task.start()
        
        
    def file_save(self,data,filename):
        default_dir =""
        default_filename = os.path.join(default_dir, filename)
        filepath, _ = QFileDialog.getSaveFileName(self, "Save file", default_filename, "(*.dat)")
        
        if filepath:
            np.savetxt(filepath, np.column_stack(data),delimiter='\t',header='Samples per channel: ' + str(sr) +'\nX\tY\tSUM')
        
        self.daq.task.stop()
        self.finished.emit()
    
    def start(self):
        self.flag=True
        
    def stop(self):
        self.daq.task.stop()
        self.daq.task.close()
        self.flag=False
               
qapp = QCoreApplication.instance()
if qapp is None:
    qapp = QApplication(sys.argv)
    
if __name__ == "__main__": 
    #start the widget
    ui = calibration_mode()
    #show the widget
    ui.show()
    #start the events loop
    qapp.exec_()

Stop Tracking
800


C:\Users\jmfo1\Anaconda3\envs\venvj\lib\site-packages\nidaqmx\task.py:97: ResourceWarning: Task of name "_unnamedTask<8>" was not explicitly closed before it was destructed. Resources on the task device may still be reserved.
  warnings.warn(


DaqError: Task specified is invalid or does not exist.
Status Code: -200088